<a href="https://colab.research.google.com/github/Visionaryofthefuture/Machine-Learning/blob/main/Torch_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset = datasets.MNIST(root='mnist', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='mnist', train=False, download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=False)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1, 1)  # 28x28 -> 28x28
        self.conv2 = nn.Conv2d(32, 64, 3, 1, 1) # 28x28 -> 28x28
        self.pool = nn.MaxPool2d(2, 2)          # 28x28 -> 14x14
        self.fc1 = nn.Linear(64 * 14 * 14, 128) # Adjusted based on pooling
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 14 * 14)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

net = Net().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

epochs = 10
for epoch in range(epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 50 == 49:
            correct = 0
            total = 0
            with torch.no_grad():
                for data in testloader:
                    images, labels = data
                    images, labels = images.to(device), labels.to(device)
                    outputs = net(images)
                    _, predicted = torch.max(outputs, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()

            accuracy = 100 * correct / total
            print(f"Epoch [{epoch + 1}/{epochs}], Step [{i + 1}], Loss: {running_loss / 50:.4f}, Accuracy: {accuracy:.2f}%")
            running_loss = 0.0

print("Finished Training")


Using device: cuda
Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:02<00:00, 4596402.55it/s]


Extracting mnist/MNIST/raw/train-images-idx3-ubyte.gz to mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 133702.97it/s]


Extracting mnist/MNIST/raw/train-labels-idx1-ubyte.gz to mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:06<00:00, 273246.52it/s]


Extracting mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 4566282.06it/s]


Extracting mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to mnist/MNIST/raw

Epoch [1/10], Step [50], Loss: 2.2487, Accuracy: 40.18%
Epoch [1/10], Step [100], Loss: 1.9985, Accuracy: 67.00%
Epoch [1/10], Step [150], Loss: 1.3125, Accuracy: 80.52%
Epoch [1/10], Step [200], Loss: 0.6827, Accuracy: 85.47%
Epoch [1/10], Step [250], Loss: 0.4952, Accuracy: 87.56%
Epoch [1/10], Step [300], Loss: 0.4450, Accuracy: 86.25%
Epoch [1/10], Step [350], Loss: 0.4248, Accuracy: 87.68%
Epoch [1/10], Step [400], Loss: 0.3824, Accuracy: 89.88%
Epoch [1/10], Step [450], Loss: 0.3540, Accuracy: 88.78%
Epoch [2/10], Step [50], Loss: 0.3641, Accuracy: 90.87%
Epoch [2/10], Step [100], Loss: 0.3284, Accuracy: 91.41%
Epoch [2/10], Step [150], Loss: 0.3370, Accuracy: 90.65%
Epoch [2/10], Step [200], Loss: 0.3175, Accuracy: 91.42%
Epoch [2/10], Step [250], Loss: 0.3073, Accuracy: 90.66%
Epoch [2/10], Step [300], Loss: 0.3001, Accuracy: 88.89%
Epoch [2/10], Step [350], Loss: 0.3084, Accuracy: 92.07%
Epoch [2/10], Ste

In [2]:
path = "./mnist_cnn_net.pth"
torch.save(net.state_dict(), path)

In [13]:
import torch

# Assuming the rest of the code above is already in place

# Load test data
dataiter = iter(testloader)
images, labels = next(dataiter)

# Load the saved model state
net.load_state_dict(torch.load(path))
net.eval()  # Set the model to evaluation mode

# Move images and labels to the device
images = images.to(device)
labels = labels.to(device)

# Make predictions
outputs = net(images)
_, predicted = torch.max(outputs, 1)

# Print predictions
print('Predicted: ', ' '.join(f'{predicted[j].item()}' for j in range(123)))
print('Actual:    ', ' '.join(f'{labels[j].item()}' for j in range(123)))
for i in range(len(predicted)):
  if predicted[i]!= labels[i]:
    print(f"Predicted: {predicted[i].item()}, Actual: {labels[i].item()}")

Predicted:  7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4 9 6 6 5 4 0 7 4 0 1 3 1 3 4 7 2 7 1 2 1 1 7 4 2 3 5 1 2 4 4 6 3 5 5 6 0 4 1 9 5 7 8 9 3 7 4 6 4 3 0 7 0 2 9 1 7 3 2 9 7 7 6 2 7 8 4 7 3 6 1 3 6 9 3 1 4 1 7 6 9 6 0 5 4 9 9 2 1 9 4 8 7 3 9 7 9 4 4 9 2 5 4 7
Actual:     7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4 9 6 6 5 4 0 7 4 0 1 3 1 3 4 7 2 7 1 2 1 1 7 4 2 3 5 1 2 4 4 6 3 5 5 6 0 4 1 9 5 7 8 9 3 7 4 6 4 3 0 7 0 2 9 1 7 3 2 9 7 7 6 2 7 8 4 7 3 6 1 3 6 9 3 1 4 1 7 6 9 6 0 5 4 9 9 2 1 9 4 8 7 3 9 7 4 4 4 9 2 5 4 7
Predicted: 9, Actual: 4
